# Location Reconciliation

This notebook can generate a Zone de Santé csv reconcililation file. The file will have a row for each case in the database, where a misspelling of a zone was detected.

In [ ]:
import django
from django.db import connection
import pandas as pd

django.setup()

## Zone mapping

This is a mapping from correct spelling to misspellings. It was generated by similarity comparing strings and manual mapping of zones that were not automatically found. Currently there are still more than 400 zones in about 2k records that cannot be mapped.

In [ ]:
MAPPING = {
 'aba': [],
 'abuzi': [],
 'adi': [],
 'adja': [],
 'aketi': [],
 'alimbongo': [],
 'alunguli': [],
 'ango': [],
 'angumu': [],
 'ankoro': [],
 'ariwara': [],
 'aru': [],
 'aungba': [],
 'bafwagbobgo': [],
 'bafwasende': [],
 'bagata': [],
 'bagira-kasha': [],
 'baka (fac)': [],
 'bambo': [],
 'bambu': [],
 'banalia': [],
 'bandalungwa': [],
 'bandjwo moke': ['bandzjow moke',
  'bandzow moke',
  'banow moke',
  'banzaw moke',
  'banzon moke',
  'banzow',
  'banzow-moke',
  'banzow moke',
  'banzow - moke',
  'banzow mokw',
  'bandjow moke'],
 'bandundu': ['bdd'],
 'banga lubaka': [],
 'bangabola': ['banga-bola', 'banga bola'],
 'barumbu': [],
 'basali': [],
 'basankusu': [],
 'basoko': [],
 'befale': [],
 'bena dibele': ['benda dibele'],
 'bena leka': [],
 'benatshiadi': ['bena tshiadi'],
 'bengamisa': [],
 'beni': [],
 'bibanga': ['biabanga',
  'biabnga',
  'bibanda',
  'bibango',
  'bibenga',
  'bibnaga',
  'bibungu'],
 'biena': [],
 'bikoro': [],
 'bili': [],
 'bilomba': [],
 'bimpemba': [],
 'binga': ['biga'],
 'binza (rutshuru)': [],
 'binza-météo': [],
 'binza-ozone': [],
 'birambizo': [],
 'biringi': [],
 'biyela': [],
 'bobozo': [],
 'boende': [],
 'boga': [],
 'bogosenubia': ['bogose',
  'bogose-nubea',
  'bogose nubea',
  'bogose - nubea',
  'bogose- nubea',
  'bogosse nubea'],
 'boko': [],
 'boko-kivulu': [],
 'bokonzi': [],
 'bokoro': [],
 'bokungu': [],
 'bolenge': [],
 'bolobo': [],
 'bolomba': [],
 'boma': [],
 'boma bungu': ['boma-bungu', 'boma bunga'],
 'boma mangbetu': [],
 'bominenge': ['bomenenge', 'bominege', 'bominenga'],
 'bomongo': [],
 'bondo': [],
 'bongandanga': [],
 'bonzola': [],
 'bosobe': [],
 'bosobolo': ['bosobole', 'bosoboloe'],
 'bosomanzi': [],
 'bosomondanda': [],
 'bosondjo': [],
 'boto': [],
 'budjala': [],
 'bukama': [],
 'bulape': [],
 'bulu': [],
 'bulungu': [],
 'bumba': [],
 'bumbu': [],
 'bunia': [],
 'bunkeya': [],
 'bunkonde': [],
 'bunyakiri': [],
 'busanga': [],
 'businga': [],
 'buta': [],
 'butembo': [],
 'butumba': [],
 'bwamanda': [],
 'cilundu': ['cilindu', 'citumbu', 'citundu'],
 'damas': [],
 'dekese': [],
 'demba': [],
 'dibaya': [],
 'dibindi': ['dibinda'],
 'dikungu ': ['dikungu',
  'dikungi',
  'dikungo',
  'dikumbu',
  'dikongo',
  'dulungu'],
 'dilala': [],
 'dilolo': [],
 'dingila': [],
 'diulu': [],
 'djalo djeka': [],
 'djolu': [],
 'djombo': [],
 'djuma': ['djuna'],
 'doruma': [],
 'drodro': [],
 'dungu': [],
 'faradja': [],
 'fataki': [],
 'ferekeni': [],
 'feshi': [],
 'fizi': [],
 'fungurume': [],
 'gandajika': [],
 'gbadolite': ['gbadolitegbado'],
 'gemena': [],
 'gethy': [],
 'goma': [],
 'gombari': [],
 'gombe': [],
 'gombe-matadi': ['gombe matadi'],
 'gungu': [],
 "hauts plateaux d'uvira": [],
 'ibanda': [],
 'iboko': [],
 'idiofa': ['idiopfa'],
 'idjwi': [],
 'ikela': [],
 'ilebo': [],
 'inga': [],
 'ingende': [],
 'inongo': [],
 'ipamu': [],
 'irebu': [],
 'isangi': [],
 'isiro': [],
 'itebero': [],
 'itombwe': [],
 'jiba': [],
 'kabalo': [],
 'kabambare': [],
 'kabare': [],
 'kabeya kamuanga': ['kabeya- kamuanga',
  'kabeya-kamuanga',
  'kabeya',
  'kabeya-kam',
  'kabeya kamuanbga',
  'kabeya-kamunga',
  'kabeya kamunga',
  'kabeya kamwanga'],
 'kabinda': [],
 'kabondo': [],
 'kabondo diamba': [],
 'kabongo': [],
 'kadutu': [],
 'kafakumba': [],
 'kafubu': [],
 'kahemba': [],
 'kailo': [],
 'kajiji': [],
 'kakenge': [],
 'kalamba': [],
 'kalambayi kabanga': ['kalambayi - kabanga',
  'kalambayi-kabanga',
  'kalambayi/ kabangakalambay',
  'kalambayi',
  'kalambayi k.',
  'kalambayi kab',
  'kalambayi kabamba',
  'kalambayi kabange',
  'kalambayi kabango',
  'kalambayi kabuanga',
  'kalambayi kasongo',
  'kalampayi',
  'kal kab',
  'kal kabanga',
  'kal. kabanga',
  'kal. kabangu',
  'kal kasanga',
  'kal kubanga',
  'kabanda',
  'kabanga'],
 'kalamu i': ['kalamu1', 'kalamu 1'],
 'kalamu ii': ['kalamu 2'],
 'kalehe ': [],
 'kalemie': [],
 'kalenda': [],
 'kalima': [],
 'kalole': [],
 'kalomba': [],
 'kalonda est': [],
 'kalonda ouest': [],
 'kalonge': [],
 'kalunguta': [],
 'kamalondo': [],
 'kamana': [],
 'kamango': [],
 'kambala': [],
 'kambove': [],
 'kamina': [],
 'kamituga': [],
 'kamji': ['kamayi', 'kamiji'],
 'kamonya': [],
 'kampemba': [],
 'kampene': [],
 'kamwesha': [],
 'kananga': [],
 'kanda kanda': [],
 'kangu': [],
 'kaniama': [],
 'kaniola': [],
 'kansele': [],
 'kansimba': [],
 'kanzala': [],
 'kanzenze': [],
 'kapanga': [],
 'kapolowe': [],
 'karawa': [],
 'karisimbi (goma)': [],
 'kasa-vubu': ['kasa vubu'],
 'kasaji': [],
 'kasansa': [],
 'kasenga': [],
 'kasongo': [],
 'kasongo lunda': [],
 'katako kombe': [],
 'katana ': [],
 'katende': ['katende`'],
 'katoka': [],
 'katoyi': [],
 'katuba': [],
 'katwa': [],
 'kayamba': [],
 'kayna': [],
 'kaziba': [],
 'kenge': [],
 'kenya': [],
 'kibirizi': [],
 'kibombo': ['kilombe', 'kilombo', 'kidombo', 'kibembo', 'kikombo'],
 'kibua': [],
 'kibunzi': [],
 'kikimi': [],
 'kikongo': [],
 'kikula': [],
 'kikwit nord': ['kikwit nona'],
 'kikwit sud': ['kikwit-sud'],
 'kilela balanda': [],
 'kilo': [],
 'kilwa': [],
 'kimbanseke': [],
 'kimbau': [],
 'kimbi lulenge': [],
 'kimpangu': [],
 'kimpese': [],
 'kimputu': [],
 'kimvula': [],
 'kinda': [],
 'kindu': [],
 'kingabwa': [],
 'kingandu': [],
 'kingasani': [],
 'kinkondja': [],
 'kinkonzi': [],
 'kinshasa': [],
 'kintambo': [],
 'kipushi': [],
 'kiri': [],
 'kirotshe': [],
 'kisandji': [],
 'kisanga': [],
 'kisantu': [],
 'kisenso': [],
 'kitangwa': [],
 'kitenda': [],
 'kitenge': [],
 'kitona': [],
 'kitutu': [],
 'kiyambi': [],
 'kizu': [],
 'kokolo': [],
 'kole': [],
 'komanda': [],
 'kongolo': [],
 'koshibanda': [],
 'kowe (fac)': [],
 'kuimba': [],
 'kunda': [],
 'kungu': [],
 'kwamouth': [],
 'kwilu-ngongo': ['kwilu - ngongo', 'kwilu ngongo'],
 'kyondo': [],
 'laybo': [],
 'lemba': [],
 'lemera': [],
 'libenge': [],
 'likasi': [],
 'likati': [],
 'lilanga bobanga': [],
 'limeté': [],
 'linga': [],
 'lingomo': [],
 'lingwala': [],
 'lisala': [],
 'lita': [],
 'lodja': [],
 'logo': [],
 'loko': [],
 'lokolela': [],
 'lolo': [],
 'lolwa': [],
 'lomela': [],
 'lotumbe': [],
 'lowa': [],
 'lualaba': [],
 'luambo': [],
 'lubao': [],
 'lubero': [],
 'lubilanji': [],
 'lubondaie': [],
 'lubudi': [],
 'lubumbashi': [],
 'lubunga': [],
 'lubutu': [],
 'ludimbi lukula': [],
 'luebo': [],
 'luiza': [],
 'lukafu': [],
 'lukelenge': ['lukalenga',
  'lukalenge',
  'lukalengi',
  'lukanga',
  'lukelenga',
  'lukelnege',
  'lukenlenge',
  'lukhenge',
  'lumelenge'],
 'lukonga': [],
 'lukula': [],
 'lulingu': [],
 'luozi': [],
 'luputa': [],
 'lusambo': ['lusamba', 'lusambu'],
 'lusanga': ['lusango'],
 'lusangi': [],
 'lwamba': [],
 'mabalako': [],
 'mahagi': [],
 'makala': [],
 'makanza': [],
 'makiso-kisangani': [],
 'makoro': [],
 'makota': [],
 'malemba nkulu': [],
 'maluku i': ['maluku 1', 'maluku1'],
 'maluku ii': ['maluku 2'],
 'mambasa': [],
 'mampoko': [],
 'mandima': [],
 'mangala': [],
 'mangembo': [],
 'mangobo': [],
 'manguredjipa': [],
 'manika': [],
 'manono': [],
 'masa': [],
 'masereka': [],
 'masi-manimba': ['masi manimba'],
 'masina i': [],
 'masina ii': [],
 'masisi': [],
 'masuika': [],
 'matadi': [],
 'matete': [],
 'mawuya': [],
 'mbandaka': [],
 'mbanza-ngungu': ['mbanza ngungu'],
 'mbaya': [],
 'mbulula': [],
 'miabi': [],
 'mikalayi': [],
 'mikope': [],
 'mimia': [],
 'minembwe': [],
 'minga': [],
 'minova': [],
 'miti - murhesa': [],
 'mitwaba': [],
 'moanza': [],
 'moba': [],
 'mobayi mbongo': [],
 'mokala': [],
 'mompono': [],
 'mondombe': [],
 'monga': [],
 'mongbwalu': [],
 'monieka': [],
 'monkoto': [],
 'mont-ngafula i': ['mont ngafula 1', 'mt ngafula 1', 'mt ngafula'],
 'mont-ngafula ii': ['mont ngafula 2', 'mont ngafula w2', 'mt ngafula 2'],
 'mosango': [],
 'mpokolo': [],
 'muanda': [],
 'mubumbano': [],
 'muene ditu': [],
 'mufunga sampwe': [],
 'mukanga': [],
 'mukedi': [],
 'mukumbi': [],
 'mulongo': [],
 'mulumba': [],
 'mulungu': [],
 'mumbunda': [],
 'mungindu': [],
 'mushenge': ['mushenga', 'mushengo'],
 'mushie': [],
 'musienene': [],
 'mutena': [],
 'mutoto': [],
 'mutshatsha': [],
 'mutwanga': [],
 'muya': [],
 'mwana': [],
 'mweka': [],
 'mwela-lembwa': [],
 'mwenga': [],
 'mweso': [],
 'mwetshi': [],
 'ndage': ['ndange', 'ngabe'],
 'ndekesha': [],
 'ndesha': [],
 'ndjili': ['n`djili', "n'djili"],
 'ndjoko mpunda': [],
 'ngaba': [],
 'ngidinga': [],
 'ngiri-ngiri': ['ngiri ngiri'],
 'nia-nia': [],
 'niangara': [],
 'nioki': [],
 'nizi': [],
 'nsele': [],
 'nselo': [],
 'nsona-pangu': ['nsona panguns. mpangu',
  'nsona mpanga',
  'nsona mpangu',
  'nsona - mpangu',
  'nsona mpanzu'],
 'ntandembelo': ['ntadembelo'],
 'ntondo': ['ntanda'],
 'nundu ': [],
 'nyakunde': [],
 'nyanga': [],
 'nyangezi': [],
 'nyantende': [],
 'nyarambe': [],
 'nyemba': [],
 'nyiragongo': [],
 'nyunzu': [],
 'nzaba': [],
 'nzanza': ['nzinzi'],
 'obokote': [],
 'oicha': [],
 'omodjadi': [],
 'opala': [],
 'opienge': [],
 'oshwe': ['oswe'],
 'ototo': [],
 'panda': [],
 'pangi': [],
 'pania mutombo': ['panya mat', 'panya mut', 'panya mutombo'],
 'panzi': [],
 'pawa': [],
 'pay kongila': ['pay-kongila', 'pay-k', 'pay-kongola'],
 'pendjwa': ['pendjua'],
 'pimu': [],
 'pinga': [],
 'poko': [],
 'police ': [],
 'popokabaka': [],
 'punia': [],
 'pweto': [],
 'rethy': [],
 'rimba': [],
 'rungu': [],
 'rutshuru': [],
 'ruzizi': [],
 'rwampara': [],
 'rwanguba': [],
 'rwashi': [],
 'sakania': [],
 'salamabila': ['salamarila', 'salambila'],
 'samba': [],
 'sandoa': [],
 'sekebanza': ['seke-banza', 'seke banza', 'seke- banza', 'sake banza'],
 'selembao': [],
 'shabunda centre ': [],
 'sia': [],
 'sona-bata': [],
 'songa': [],
 'tandala': ['tandale', 'tdl', 'tdla'],
 'tchiomia': [],
 'tembo': [],
 'titule': [],
 'tshela': ['tshiela'],
 'tshiamilemba': [],
 'tshibala': ['tshibila'],
 'tshikaji': [],
 'tshikapa': [],
 'tshikula': ['tshikila', 'tshikulu'],
 'tshilenge': ['tshilengi'],
 'tshitenge': ['tshiteng', 'tshitenga', 'tshitinga'],
 'tshitshimbi': ['tshishimba', 'tshishimbi'],
 'tshofa': [],
 'tshopo': [],
 'tshudi loto': [],
 'tshumbe': ['tshumbo'],
 'tunda': ['tundja'],
 'ubundu': [],
 'uvira': [],
 'vaku': [],
 'vanga': [],
 'vanga kete': [],
 'vangu (fac)': [],
 'viadana': [],
 'vuhovi': [],
 'walikale': [],
 'walungu': [],
 'wamba': [],
 'wamba lwadi': [],
 'wangata': [],
 'wanierukula': [],
 'wapinda': [],
 'wasolo': [],
 'watsa': [],
 'wema': [],
 'wembo nyama': ['wembo-nyama',
  'wembanyama',
  'wembenyama',
  'wembonama',
  'wembonya',
  'wembonyama'],
 'wikong': [],
 'yabaondo': [],
 'yahisuli': [],
 'yahuma': [],
 'yakoma': [],
 'yakusu': [],
 'yaleko': [],
 'yalifafo': [],
 'yalimbongo': [],
 'yamaluka': [],
 'yambuku': [],
 'yamongili': [],
 'yangala': [],
 'yasa-bonga': ['yasa bonga', 'bonga yasa'],
 'yumbi': [],
 'zongo': []
}

from string import capwords

# Create a mapping for each misspelling that can be used in sql
FLAT_MAPPING = []
for (name, spellings) in MAPPING.items():
    for spelling in spellings:
        # Capitalize the names to follow the naming scheme that is already in use
        FLAT_MAPPING.append((capwords(name), capwords(spelling)))
FLAT_MAPPING


## Some sql queries for reuse

CTEs for querying the zones and cases

In [ ]:
misspelled_zones = ','.join("($${}$$, $${}$$)".format(a, b) for (a, b) in FLAT_MAPPING)
correct_zones = ','.join("($${}$$)".format(k) for k in MAPPING.keys())

CTEs = {
'correct_zones': '''
  correct_zones as (
      select zs
      from (values {})
      as z(zs)
  )
'''.format(correct_zones),

'misspelled_zones': '''
  misspelled_zones as (
    select zs, zs_spelling
    from (values {})
    as z(zs, zs_spelling)
  )
'''.format(misspelled_zones),

'misspelled_cases': '''
  misspelled_cases as (
    select distinct id, c.document_id, z.zs
    from cases_case as c
    inner join misspelled_zones as z
    on lower(c."ZS") = lower(z.zs_spelling)
  )
''',

'correct_cases': '''
  correct_cases as (
    select distinct id, c.document_id, z.zs
    from cases_case as c
    inner join correct_zones as z
    on lower(c."ZS") = lower(z.zs)
  )
''',
    
'unknown_zones': '''
  unknown_zones as (
    select distinct "ZS" as zs
    from cases_case A
    where not exists (select * from correct_cases B where A.id = B.id)
      and not exists (select * from misspelled_cases B where A.id = B.id)
  )
'''
}

## Get the number of mappable and other cases

In [ ]:
with connection.cursor() as c:
    c.execute('''
        select set_limit(1);
        with
          {correct_zones},
          {misspelled_zones},
          {correct_cases},
          {misspelled_cases}
          select
              (select count(*) from cases_case),
              (select count(*) from correct_cases),
              (select count(*) from misspelled_cases)
          
    '''.format(**CTEs))
    results = c.fetchall()
    [(total, correct, misspelled)] = results
    print('Total:', total, 'Correct:', correct, 'Misspelled:', misspelled, 'Unknown:', total - (correct + misspelled))
    print('done.')

## Create a reconciliation csv file

The file will contain rows that have the cases `document_id` and the corrected zone name.

In [ ]:
with connection.cursor() as c:
    c.execute('''
        select set_limit(1);
        with
          {misspelled_zones},
          {misspelled_cases}
          select document_id, zs from misspelled_cases
    '''.format(**CTEs))
    results = c.fetchall()
    print('num results', len(results))
    # Write matching (document_id, ZS) tuples to a csv file
    from pandas import DataFrame
    df = DataFrame(results)
    df.to_csv('/opt/shared/location_reconciliation.csv', header=['document_id', 'ZS'], index=False)
    print('done.')

## Find zones outside the mapping

Not all zones are in the mapping. This query lists all zones which are currently not mappable.

In [ ]:
with connection.cursor() as c:
    c.execute('''
        select set_limit(1);
        with
          {correct_zones},
          {misspelled_zones},
          {correct_cases},
          {misspelled_cases},
          {unknown_zones}
          select * from unknown_zones
    '''.format(**CTEs))
    results = c.fetchall()
    print([r for (r,) in results])
    print('done.')